##Validando a SparkSession

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

##Conectando Azure ADLS Gen2 no Databricks

###Mostrando os pontos de montagem no cluster Databricks

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/datalake6d3129a9a4390b0a/gold,wasbs://gold@datalake6d3129a9a4390b0a.blob.core.windows.net,
/mnt/datalakeedda2e0be8d3ec5f/bronze,wasbs://bronze@datalakeedda2e0be8d3ec5f.blob.core.windows.net,
/mnt/datalake246d728b13561613/lading-zone,wasbs://lading-zone@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/bronze,wasbs://bronze@datalake4b6c87c48101c278.blob.core.windows.net,
/databricks-datasets,databricks-datasets,
/mnt/datalake246d728b13561613/bronze,wasbs://bronze@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalakee66472c14f9678fd/landing-zone,wasbs://landing-zone@datalakee66472c14f9678fd.blob.core.windows.net,
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/mnt/datalakeedda2e0be8d3ec5f/landing-zone,wasbs://landing-zone@datalakeedda2e0be8d3ec5f.blob.core.windows.net,
/databricks-results,databricks-results,sse-s3


###Desmontando os pontos de montagem não utilizados

In [0]:
#dbutils.fs.unmount('/mnt/datalake37e6a883b8e661af/lading-zone')
#dbutils.fs.unmount('/mnt/datalaked10514ad1cefdbcd/bronze')
#dbutils.fs.unmount('/mnt/datalaked10514ad1cefdbcd/silver')
#dbutils.fs.unmount('/mnt/datalaked10514ad1cefdbcd/gold')

### Definindo uma função para montar um ADLS com um ponto de montagem com ADLS SAS 

In [0]:
storageAccountName = "datalake4b6c87c48101c278"
storageAccountAccessKey = ""
sasToken = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-07T12:17:21Z&st=2025-06-07T04:17:21Z&spr=https&sig=BfwzYwM%2B5YR%2FBkSSyRv0Nn%2F3riHK9Wx4P%2FQoguM%2BA%2FM%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

###Montando todos os containers

In [0]:
mount_adls('landing-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

Falha An error occurred while calling o1888.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake4b6c87c48101c278/landing-zone; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake4b6c87c48101c278/landing-zone
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:135)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:1053)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.$anonfun$mount$1(DBUtilsCore.scala:1079)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:657)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(Usa

###Mostrando os pontos de montagem no cluster Databricks

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/datalake6d3129a9a4390b0a/gold,wasbs://gold@datalake6d3129a9a4390b0a.blob.core.windows.net,
/mnt/datalakeedda2e0be8d3ec5f/bronze,wasbs://bronze@datalakeedda2e0be8d3ec5f.blob.core.windows.net,
/mnt/datalake246d728b13561613/lading-zone,wasbs://lading-zone@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/bronze,wasbs://bronze@datalake4b6c87c48101c278.blob.core.windows.net,
/mnt/datalake4b6c87c48101c278/gold,wasbs://gold@datalake4b6c87c48101c278.blob.core.windows.net,
/databricks-datasets,databricks-datasets,
/mnt/datalake246d728b13561613/bronze,wasbs://bronze@datalake246d728b13561613.blob.core.windows.net,
/mnt/datalakee66472c14f9678fd/landing-zone,wasbs://landing-zone@datalakee66472c14f9678fd.blob.core.windows.net,
/databricks/mlflow-tracking,databricks/mlflow-tracking,sse-s3
/mnt/datalake4b6c87c48101c278/silver,wasbs://silver@datalake4b6c87c48101c278.blob.core.windows.net,


### Mostrando todos os arquivos da camada landing-zone

In [0]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/landing-zone"))

path,name,size,modificationTime
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/customers_202506062219.csv,customers_202506062219.csv,8910522,1749270372000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/geolocation_202506062219.csv,geolocation_202506062219.csv,59657432,1749270426000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/leads_closed_202506062219.csv,leads_closed_202506062219.csv,178083,1749270369000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/leads_qualified_202506062219.csv,leads_qualified_202506062219.csv,747223,1749270369000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/order_items_202506062219.csv,order_items_202506062219.csv,15680605,1749270362000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/order_payments_202506062219.csv,order_payments_202506062219.csv,5866607,1749270360000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/order_reviews_202506062219.csv,order_reviews_202506062219.csv,14700106,1749270374000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/orders_202506062219.csv,orders_202506062219.csv,18738950,1749270377000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/product_category_name_translation_202506062219.csv,product_category_name_translation_202506062219.csv,2616,1749270358000
dbfs:/mnt/datalake4b6c87c48101c278/landing-zone/products_202506062219.csv,products_202506062219.csv,2870036,1749270366000


###Gerando um dataframe para cada arquivo a partir dos arquivos CSV gravado no container landing-zone do Azure Data Lake Storage

In [0]:
df_customers   = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/customers_202506062219.csv")
df_geolocation = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/geolocation_202506062219.csv")
df_leads_closed  = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/leads_closed_202506062219.csv")
df_leads_qualified  = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/leads_qualified_202506062219.csv")
df_order_items = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/order_items_202506062219.csv")
df_order_payments  = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/order_payments_202506062219.csv")
df_order_reviews = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/order_reviews_202506062219.csv")
df_orders = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/orders_202506062219.csv")
df_product_category_name_translation = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/product_category_name_translation_202506062219.csv")
df_products = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/products_202506062219.csv")
df_sellers = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/sellers_202506062219.csv")


### Adicionando metadados de data e hora de processamento e nome do arquivo de origem

In [0]:
from pyspark.sql.functions import current_timestamp, lit

df_customers = df_customers.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("customers_202506062219.csv"))
df_geolocation = df_geolocation.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("geolocation_202506062219.csv"))
df_leads_closed = df_leads_closed.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("leads_closed_202506062219.csv"))
df_leads_qualified = df_leads_qualified.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("leads_qualified_202506062219.csv"))
df_order_items = df_order_items.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("order_items_202506062219.csv"))
df_order_payments = df_order_payments.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("order_payments_202506062219.csv"))
df_order_reviews = df_order_reviews.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("order_reviews_202506062219.csv"))
df_orders = df_orders.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("orders_202506062219.csv"))
df_product_category_name_translation = df_product_category_name_translation.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("product_category_name_translation_202506062219.csv"))
df_products = df_products.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("products_202506062219.csv"))
df_sellers = df_sellers.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("sellers_202506062219.csv"))


###Salvando os dataframes em delta lake (formato de arquivo) no data lake (repositorio cloud)

In [0]:
df_customers.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/customers")
df_geolocation.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/geolocation")
df_leads_closed.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/leads_closed")
df_leads_qualified.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/leads_qualified")
df_order_items.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/order_items")
df_order_payments.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/order_payments")
df_order_reviews.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/order_reviews")
df_orders.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/orders")
df_product_category_name_translation.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/product_category_name_translation")
df_products.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/products")
df_sellers.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/sellers")

###Verificando os dados gravados em delta na camada bronze

In [0]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze/"))

path,name,size,modificationTime
dbfs:/mnt/datalake4b6c87c48101c278/bronze/customers/,customers/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/geolocation/,geolocation/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/leads_closed/,leads_closed/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/leads_qualified/,leads_qualified/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/order_items/,order_items/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/order_payments/,order_payments/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/order_reviews/,order_reviews/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/orders/,orders/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/product_category_name_translation/,product_category_name_translation/,0,0
dbfs:/mnt/datalake4b6c87c48101c278/bronze/products/,products/,0,0


### Lendo um exemplo de um delta lake para validar a existencia dos dados e das colunas do metadados

In [0]:
spark.read.format('delta').load(f'/mnt/{storageAccountName}/bronze/products').limit(10).display()

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,data_hora_bronze,nome_arquivo
1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,60.0,745.0,1.0,200.0,38.0,5.0,11.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
732bd381ad09e530fe0a5f457d81becb,cool_stuff,56.0,1272.0,4.0,18350.0,70.0,24.0,44.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,56.0,184.0,2.0,900.0,40.0,8.0,40.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
37cc742be07708b53a98702e77a21a02,eletrodomesticos,57.0,163.0,1.0,400.0,27.0,13.0,17.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
8c92109888e8cdf9d66dc7e463025574,brinquedos,36.0,1156.0,1.0,600.0,17.0,10.0,12.0,2025-06-07T04:29:29.911+0000,products_202506062219.csv
